# 6.3 语言模型数据集（周杰伦专辑歌词）

In [2]:
import torch
import random
import zipfile

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.__version__)
print(device)

1.3.1
cpu


## 6.3.1 读取数据集

In [3]:
with zipfile.ZipFile('../../data/jaychou_lyrics.txt.zip') as zin:
    with zin.open('jaychou_lyrics.txt') as f:
        corpus_chars = f.read().decode('utf-8')
corpus_chars[:40]

'想要有直升机\n想要和你飞到宇宙去\n想要和你融化在一起\n融化在宇宙里\n我每天每天每'

In [4]:
corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
corpus_chars = corpus_chars[0:10000]
corpus_chars[0:40]# 每一个字和空格都算一个字符

'想要有直升机 想要和你飞到宇宙去 想要和你融化在一起 融化在宇宙里 我每天每天每'

## 6.3.2 建立字符索引

In [15]:
idx_to_char = list(set(corpus_chars)) # set返回一个元素不重复的集合{}，注意不是字典,一般set过后做一个list操作便于索引
char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
vocab_size = len(char_to_idx)
print(set(corpus_chars))
print(idx_to_char)
print(vocab_size)
print(char_to_idx)

{'呵', '言', '甩', '城', '封', '经', '什', '祭', '昏', '埋', '九', '蜜', '明', '怕', '截', 'o', '待', '便', '常', '敌', '何', '丁', '排', '场', '界', '手', '菸', '病', '颁', '张', '阻', '晴', '样', '奈', '简', '刚', '兮', '果', '客', '蛇', '狠', '彻', '江', '腿', '型', '忠', '腔', '征', '法', '斑', ' ', '落', '条', '模', '忙', '队', '练', '粥', '印', '吸', '漂', '藤', '容', '找', '每', '年', '汉', '移', '歉', '医', '辛', '圈', '古', '怯', '争', '优', '奏', '弥', '弄', '闭', '环', '银', '纪', '子', '别', '注', '彷', '荒', '能', '中', '数', '伯', '蝴', '莫', '葛', '师', '来', '寂', '深', '盯', '背', '怎', '真', '多', '擅', '廉', '动', '戏', '义', '融', '切', '边', '防', '蝙', '熬', '脉', '声', '逃', '妥', '力', '奔', '逅', '他', '而', '肩', '底', '望', '游', '哪', '她', '湖', '在', '被', '缝', '殿', '性', '成', '停', '除', '吗', '池', '醉', '解', '都', '元', '形', '嘴', '云', '阳', '刺', '记', '童', '十', '带', '土', '学', '据', '语', '很', '蒙', '彩', '袭', '认', '后', '廓', '假', '够', '载', '周', '饿', '悬', '晰', '颗', '滩', '水', '就', '林', '怨', '亮', '题', '情', '蜡', '牛', '掩', '育', '努', '屈', '社', '啦', '凯', '猫', '儿', '字', '凝', '馆', '见', '忆', '邂', '阵', '让',

In [12]:
corpus_indices = [char_to_idx[char] for char in corpus_chars]#因为是char in corpus_chars所以才是这样有效的
sample = corpus_indices[:33]
print('chars:', ''.join([idx_to_char[idx] for idx in sample]))
print('indices:', sample) #空格也是字符 在本例子中索引为133

chars: 想要有直升机 想要和你飞到宇宙去 想要和你融化在一起 融化在宇宙里
indices: [350, 903, 589, 768, 357, 329, 50, 350, 903, 914, 1007, 827, 443, 476, 802, 523, 50, 350, 903, 914, 1007, 109, 219, 131, 587, 229, 50, 109, 219, 131, 476, 802, 611]


## 6.3.3 时序数据的采样
### 6.3.3.1 随机采样

In [22]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def data_iter_random(corpus_indices, batch_size, num_steps, device=None):
    # 减1是因为输出的索引x是相应输入的索引y加1
    num_examples = (len(corpus_indices) - 1) // num_steps # 一个时间步有多少语料要输入
    epoch_size = num_examples // batch_size # 表示一个epoch有要训练多少个batch_size
    example_indices = list(range(num_examples))
    random.shuffle(example_indices)

    # 返回从pos开始的长为num_steps的序列
    def _data(pos):
        return corpus_indices[pos: pos + num_steps]
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    for i in range(epoch_size):
        # 每次读取batch_size个随机样本
        i = i * batch_size
        batch_indices = example_indices[i: i + batch_size]
        X = [_data(j * num_steps) for j in batch_indices]
        Y = [_data(j * num_steps + 1) for j in batch_indices]
        yield torch.tensor(X, dtype=torch.float32, device=device), torch.tensor(Y, dtype=torch.float32, device=device)

In [23]:
my_seq = list(range(30))
for X, Y in data_iter_random(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')

X:  tensor([[ 6.,  7.,  8.,  9., 10., 11.],
        [12., 13., 14., 15., 16., 17.]]) 
Y: tensor([[ 7.,  8.,  9., 10., 11., 12.],
        [13., 14., 15., 16., 17., 18.]]) 

X:  tensor([[18., 19., 20., 21., 22., 23.],
        [ 0.,  1.,  2.,  3.,  4.,  5.]]) 
Y: tensor([[19., 20., 21., 22., 23., 24.],
        [ 1.,  2.,  3.,  4.,  5.,  6.]]) 



### 6.3.3.2 相邻采样

In [17]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def data_iter_consecutive(corpus_indices, batch_size, num_steps, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_indices = torch.tensor(corpus_indices, dtype=torch.float32, device=device)
    data_len = len(corpus_indices)
    batch_len = data_len // batch_size
    indices = corpus_indices[0: batch_size*batch_len].view(batch_size, batch_len)
    epoch_size = (batch_len - 1) // num_steps
    for i in range(epoch_size):
        i = i * num_steps
        X = indices[:, i: i + num_steps]
        Y = indices[:, i + 1: i + num_steps + 1]
        yield X, Y

In [18]:
for X, Y in data_iter_consecutive(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')

X:  tensor([[ 0.,  1.,  2.,  3.,  4.,  5.],
        [15., 16., 17., 18., 19., 20.]]) 
Y: tensor([[ 1.,  2.,  3.,  4.,  5.,  6.],
        [16., 17., 18., 19., 20., 21.]]) 

X:  tensor([[ 6.,  7.,  8.,  9., 10., 11.],
        [21., 22., 23., 24., 25., 26.]]) 
Y: tensor([[ 7.,  8.,  9., 10., 11., 12.],
        [22., 23., 24., 25., 26., 27.]]) 

